In [3]:
import pandas as pd
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True # also works
PATH = 'C:\\Users\\devna\\Ecotec\\chromedriver.exe'
driver = webdriver.Chrome(PATH, options=chrome_options)

In [28]:
# Scraping Function

def scrape_table(table_type):
    table = driver.find_element_by_tag_name(f'table#teams_standard_{table_type}')
    cols = []
    header = table.find_element_by_tag_name('thead')
    header_cols = header.find_elements_by_tag_name('th')
    cols = [i.text for i in header_cols]

    # Create DF
    df = pd.DataFrame(columns = cols)

    # Get row values
    body = table.find_element_by_tag_name('tbody')
    rows = body.find_elements_by_tag_name('tr')
    for row in rows:
        team = row.find_element_by_tag_name('th')
        try:
            team_attrs = team.find_element_by_tag_name('a')
            team_name = team_attrs.get_attribute('title')
        except:
            team_name = 'LgAvg'
            
        if (team_name == 'Los Angeles Angels of Anaheim'):
            team_name = 'Los Angeles Angels'
        if (team_name == 'Florida Marlins'):
            team_name = 'Miami Marlins'
        if (team_name == 'Tampa Bay Devil Rays'):
            team_name = 'Tampa Bay Rays'

        stats = row.find_elements_by_tag_name('td')
        stat_list = [team_name]
        for stat in stats:
            stat_list.append(stat.text)

        df.loc[len(df)] = stat_list
    
    return df

In [29]:
url = 'https://www.baseball-reference.com/leagues/MLB/2021.shtml'
driver.get(url)

In [30]:
# Scrape the tables

pitching_df = scrape_table('pitching')
batting_df = scrape_table('batting')
fielding_df = scrape_table('fielding')

In [42]:
print(len(pitching_df), len(batting_df), len(fielding_df))

31 31 31


In [31]:
merged = pd.merge(batting_df, fielding_df, on='Tm').merge(pitching_df, on='Tm')
merged = merged.add_prefix('2021 ')
merged.rename(columns={'2021 Tm': 'Tm'}, inplace=True)

In [32]:
def scrape_years(year):
    
    driver.get(f'https://www.baseball-reference.com/leagues/MLB/{year}.shtml')
    
    pitching_df = scrape_table('pitching')
    batting_df = scrape_table('batting')
    fielding_df = scrape_table('fielding')
    
    merged = pd.merge(batting_df, fielding_df, on='Tm').merge(pitching_df, on='Tm')
    merged = merged.add_prefix(f'{year} ')
    merged.rename(columns={f'{year} Tm': 'Tm'}, inplace=True)
    
    return merged

In [33]:
# Scrape all years

years = [i for i in range(2005, 2021)]

for year in years:
    df = scrape_years(year)
    merged = merged.merge(df, on='Tm')
    print(year)
    
print(len(merged['Tm']))

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
31


In [34]:
merged.isnull().sum().sum()

0

In [12]:
merged.to_csv('MLB Stats.csv', index=False)

In [18]:
# Function to scrape summary stats

def scrape_exp_stats():
    table = driver.find_element_by_tag_name('table#expanded_standings_overall')

    # Columns
    cols = []
    header = table.find_element_by_tag_name('thead')
    header_cols = header.find_elements_by_tag_name('th')
    cols = [i.text for i in header_cols]
    cols[0], cols[1] = cols[1], cols[0]
    cols.pop(2)

    # Create DF
    df = pd.DataFrame(columns = cols)

    # Get row values
    body = table.find_element_by_tag_name('tbody')
    rows = body.find_elements_by_tag_name('tr')
    for row in rows:
        rank = row.find_element_by_tag_name('th').text
        try:
            team = row.find_element_by_tag_name('td')
            team_attrs = team.find_element_by_tag_name('a')
            team_name = team_attrs.get_attribute('title')
        except:
            team_name = 'LgAvg'
        
        if (team_name == 'Los Angeles Angels of Anaheim'):
            team_name = 'Los Angeles Angels'
        if (team_name == 'Florida Marlins'):
            team_name = 'Miami Marlins'
        if (team_name == 'Tampa Bay Devil Rays'):
            team_name = 'Tampa Bay Rays'

        stats = row.find_elements_by_tag_name('td')
        stat_list = [team_name, rank]
        for stat in stats:
            stat_list.append(stat.text)
        if (len(stat_list) > 2):
            stat_list.pop(2)
            stat_list.pop(2)
            df.loc[len(df)] = stat_list


    return df

In [19]:
# Scrape current year to merge with others

url = 'https://www.baseball-reference.com/leagues/MLB/2021-standings.shtml'
driver.get(url)

df = scrape_exp_stats()

df = df.add_prefix('2021 ')
df.rename(columns={'2021 Tm': 'Tm'}, inplace=True)

df

,Tm,2021 Rk,2021 G,2021 W,2021 L,2021 W-L%,2021 Strk,2021 R,2021 RA,2021 Rdiff,...,2021 Road,2021 ExInn,2021 1Run,2021 vRHP,2021 vLHP,2021 ≥.500,2021 <.500,2021 last10,2021 last20,2021 last30
0,Los Angeles Dodgers,1,21,15,6,.714,W 1,4.9,3.3,1.6,...,8-4,1-1,4-3,9-2,6-4,6-5,9-1,6-4,15-5,15-6
1,Oakland Athletics,2,21,14,7,.667,W 13,4.8,4.5,0.2,...,6-1,2-0,4-0,10-6,4-1,3-7,11-0,10-0,14-6,14-7
2,Kansas City Royals,3,19,12,7,.632,W 3,4.6,4.6,-0.1,...,4-2,1-0,5-0,7-4,5-3,2-3,10-4,7-3,12-7,12-7
3,Seattle Mariners,4,21,13,8,.619,W 1,4.5,4.2,0.3,...,7-3,4-0,6-4,10-5,3-3,8-6,5-2,7-3,12-8,13-8
4,San Francisco Giants,5,21,13,8,.619,L 1,3.6,3.2,0.4,...,6-6,1-2,5-3,8-5,5-3,5-4,8-4,6-4,13-7,13-8
5,Milwaukee Brewers,6,20,12,8,.600,W 1,4.3,3.7,0.6,...,8-3,2-1,2-2,12-8,0-0,10-4,2-4,6-4,12-8,12-8
6,Boston Red Sox,7,22,13,9,.591,L 1,5.2,4.4,0.8,...,6-1,2-1,3-2,8-6,5-3,6-4,7-5,4-6,13-7,13-9
7,Chicago White Sox,8,20,11,9,.550,W 3,5.0,4.2,0.9,...,6-6,0-2,3-2,7-8,4-1,5-4,6-5,6-4,11-9,11-9
8,Tampa Bay Rays,9,21,11,10,.524,W 1,4.9,5.0,-0.2,...,7-5,0-3,2-2,8-7,3-3,2-4,9-6,6-4,10-10,11-10
9,San Diego Padres,10,23,12,11,.522,L 1,3.7,3.3,0.3,...,7-3,0-2,2-3,9-9,3-2,4-8,8-3,4-6,9-11,12-11


In [20]:
# Scrape sum stats for past 10 years

years = [i for i in range(2005, 2021)]

new_stats = df.copy()

for year in years:
    driver.get(f'https://www.baseball-reference.com/leagues/MLB/{year}-standings.shtml')
    df = scrape_exp_stats()
    df = df.add_prefix(f'{year} ')
    df.rename(columns={f'{year} Tm': 'Tm'}, inplace=True)
    new_stats = new_stats.merge(df, on='Tm')
    print('Scraped: ', year)  

new_stats

Scraped:  2005
Scraped:  2006
Scraped:  2007
Scraped:  2008
Scraped:  2009
Scraped:  2010
Scraped:  2011
Scraped:  2012
Scraped:  2013
Scraped:  2014
Scraped:  2015
Scraped:  2016
Scraped:  2017
Scraped:  2018
Scraped:  2019
Scraped:  2020


,Tm,2021 Rk,2021 G,2021 W,2021 L,2021 W-L%,2021 Strk,2021 R,2021 RA,2021 Rdiff,...,2020 vWest,2020 Inter,2020 Home,2020 Road,2020 ExInn,2020 1Run,2020 vRHP,2020 vLHP,2020 ≥.500,2020 <.500
0,Los Angeles Dodgers,1,21,15,6,.714,W 1,4.9,3.3,1.6,...,27-13,16-4,21-9,22-8,5-1,7-5,30-12,13-5,8-5,35-12
1,Oakland Athletics,2,21,14,7,.667,W 13,4.8,4.5,0.2,...,26-14,10-10,22-10,14-14,6-1,7-6,25-21,11-3,2-4,34-20
2,Kansas City Royals,3,19,12,7,.632,W 3,4.6,4.6,-0.1,...,0-0,9-11,15-15,11-19,1-2,8-9,22-27,4-7,17-27,9-7
3,Seattle Mariners,4,21,13,8,.619,W 1,4.5,4.2,0.3,...,20-20,7-13,14-10,13-23,2-3,7-8,19-21,8-12,8-12,19-21
4,San Francisco Giants,5,21,13,8,.619,L 1,3.6,3.2,0.4,...,18-22,11-9,19-14,10-17,2-3,8-8,15-21,14-10,7-19,22-12
5,Milwaukee Brewers,6,20,12,8,.600,W 1,4.3,3.7,0.6,...,0-0,10-10,15-14,14-17,3-2,11-5,18-24,11-7,19-24,10-7
6,Boston Red Sox,7,22,13,9,.591,L 1,5.2,4.4,0.8,...,0-0,10-10,11-20,13-16,0-4,4-9,17-19,7-17,13-26,11-10
7,Chicago White Sox,8,20,11,9,.550,W 3,5.0,4.2,0.9,...,0-0,10-10,18-12,17-13,1-2,5-8,21-25,14-0,12-20,23-5
8,Tampa Bay Rays,9,21,11,10,.524,W 1,4.9,5.0,-0.2,...,0-0,13-7,20-9,20-11,4-3,14-5,31-12,9-8,21-9,19-11
9,San Diego Padres,10,23,12,11,.522,L 1,3.7,3.3,0.3,...,24-16,13-7,21-11,16-12,4-0,8-8,27-13,10-10,6-7,31-16


In [36]:
# Merge new summarys stats with previous stats

total = merged.merge(new_stats, on='Tm')

# Check NaN's
print(total.isna().sum().sum())

# Drop current year only columns
total.drop(['2021 last30', '2021 last20', '2021 last10', '2021 Strk'], axis=1, inplace=True)
total

,Tm,2021 #Bat,2021 BatAge,2021 R/G,2021 G_x,2021 PA,2021 AB,2021 R_x,2021 H_x,2021 2B,...,2020 vWest,2020 Inter,2020 Home,2020 Road,2020 ExInn,2020 1Run,2020 vRHP,2020 vLHP,2020 ≥.500,2020 <.500
0,Arizona Diamondbacks,34,29.5,5.20,20,816,707,104,159,27,...,14-26,11-9,16-14,9-21,0-3,9-9,20-24,5-11,9-15,16-20
1,Atlanta Braves,33,27.9,4.79,19,717,615,91,140,31,...,0-0,11-9,19-11,16-14,2-3,11-6,27-21,8-4,12-9,23-16
2,Baltimore Orioles,30,27.1,3.70,20,725,668,74,150,35,...,0-0,11-9,13-20,12-15,4-4,7-12,18-29,7-6,11-26,14-9
3,Boston Red Sox,29,28.1,5.18,22,824,746,114,208,51,...,0-0,10-10,11-20,13-16,0-4,4-9,17-19,7-17,13-26,11-10
4,Chicago Cubs,34,29.6,4.60,20,737,633,92,137,26,...,0-0,12-8,19-14,15-12,3-1,10-9,27-22,7-4,19-14,15-12
5,Chicago White Sox,29,28.0,5.00,20,766,677,100,174,33,...,0-0,10-10,18-12,17-13,1-2,5-8,21-25,14-0,12-20,23-5
6,Cincinnati Reds,31,29.0,5.75,20,777,684,115,174,34,...,0-0,10-10,16-13,15-16,2-0,7-8,24-23,7-6,13-17,18-12
7,Cleveland Indians,27,27.0,3.63,19,690,614,69,124,26,...,0-0,12-8,18-12,17-13,4-2,8-6,27-20,8-5,16-15,19-10
8,Colorado Rockies,27,28.2,4.40,20,714,655,88,153,32,...,17-23,9-11,12-18,14-16,1-2,9-8,16-24,10-10,9-15,17-19
9,Detroit Tigers,32,28.1,3.19,21,728,666,67,138,23,...,0-0,11-7,12-15,11-20,1-2,7-9,16-31,7-4,14-27,9-8


In [38]:
total.to_csv('MLB All Stats.csv', index=False)

In [ ]:
# Clean this file up

# Scrape all the games from 2010 to present